In [1]:
## Initial setup
import qiskit
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler
import os
from dotenv import load_dotenv
import json
import random

load_dotenv()
API_TOKEN = os.getenv("API_TOKEN")
service = QiskitRuntimeService(channel="ibm_quantum", token=API_TOKEN)
backend = service.backend(name="ibm_rensselaer")

In [2]:
''' 
trials schema:
{
"(num_vars,complexity)" : [
            {
                "job_id": string,
                "counts": counts | None,
                "input_state": binary_string, # format x0 x1 x2 ... xn 0
                "statement": string,
                "variables": [string, ...],
                "expected_result": boolean, # just the result qubit; implicitly expect first n qubits to be identical to input
            }
        ] 
}
'''

def unpack_key(key):
    return tuple(map(int, key[1:-1].split(",")))

class Trial:
    def __init__(self, num_vars, complexity, job_id, input_state, statement, variables, expected_result, counts=None):
        self.num_vars = num_vars
        self.compexity = complexity
        self.job_id = job_id
        self.input_state = input_state
        self.statement = statement
        self.variables = variables
        self.expected_result = expected_result
        self.counts = counts

    def as_dict(self):
        return {
            "job_id": self.job_id,
            "input_state": self.input_state,
            "statement": self.statement,
            "variables": self.variables,
            "expected_result": self.expected_result,
            "counts": self.counts
        }
    
    @classmethod
    def from_dict(cls, d, key):
        num_vars, complexity = unpack_key(key)
        return cls(
            num_vars = num_vars,
            complexity = complexity,
            job_id = d["job_id"],
            input_state = d["input_state"],
            statement = d["statement"],
            variables = d["variables"],
            expected_result = d["expected_result"],
            counts = d["counts"]
        )
    
    def get_key(self):
        return f"({self.num_vars},{self.compexity})"
    
    def set_counts(self, counts, trials = None):
        """ 
        call this once the job has finished running

        this updates the trials entry trials is provided and the entry exists
        """
        self.counts = counts
        if trials is not None:
            key = self.get_key()
            for trial in trials[key]:
                if trial["job_id"] == self.job_id:
                    trial["counts"] = counts

    def get_counts(self):
        """
        returns the counts for this job (waiting for them if they are not complete)
        this sets the counts if they are not already set
        """
        if self.counts is None:
            retrieved_job = service.job(self.job_id)
            counts = retrieved_job.result()[0].data.meas.get_counts()
            self.set_counts(counts)
        return self.counts
    
    def add_to_trials(self, trials):
        key = self.get_key()
        if key not in trials:
            trials[key] = []
        trials[key].append(self.as_dict())
        save_trials(trials)

    def total_expected_results(self):
        """ 
        Returns the expected binary string to be measured after the circuit is run
        Note that due to little-endian encoding of the counts, the first bit is the result bit
        and the input bits are in reverse order
        """
        result_bit = "1" if self.expected_result else "0"
        return result_bit + self.input_state[:-1]

def load_trials():
    trials = {}
    with open("benchmark_trials.json") as f:
        trials = json.load(f)
    return trials

def save_trials(trials):
    with open("benchmark_trials.json", "w") as f:
        json.dump(trials, f)

def init_trials(trials):
    from benchmark_functions import benchmark_functions
    for key in benchmark_functions.keys():
        trials[key] = trials.get(key, [])
    save_trials(trials)


In [9]:

NUM_TRIALS = 4
from benchmark_functions import *

def run_benchmark():
    trials = load_trials()
    init_trials(trials)
    for key in benchmark_functions.keys():
        num_vars, complexity = unpack_key(key)
        while len(trials[key]) < NUM_TRIALS:
            statement = benchmark_functions[key]["statement"]
            variables = benchmark_functions[key]["variables"]

            qc = qiskit.QuantumCircuit(num_vars + 1)  # +1 for result qubit

            # generate random input for each trial
            inpt = [random.randint(0, 1) for _ in range(num_vars)]
            for i, bit in enumerate(inpt):
                if bit:
                    qc.x(qc.qubits[i])

            qc.compose(eval(benchmark_functions[key]["classical_function"]).synth(registerless=False), inplace=True)
            qc.measure_all()
            qc_transpiled = qiskit.transpile(qc, backend=backend)
            sampler = Sampler(backend)
            job = sampler.run([qc_transpiled], shots=1024)

            
            expected_result = eval(benchmark_functions[key]["python_function"])(*inpt)
            
            Trial(num_vars, complexity, job.job_id(), ''.join(map(str, inpt)), statement, variables, expected_result).add_to_trials(trials)

def fetch_results():
    """ 
    Pulls the result counts from the jobs for all trials
    """
    trials = load_trials()
    for key in trials.keys():
        for _trial in trials[key]:
            trial = Trial.from_dict(_trial, key)
            counts = trial.get_counts()
            trial.set_counts(counts, trials)
    save_trials(trials)


In [7]:
run_benchmark()


In [10]:
fetch_results()